In [1]:
from mleap.analyze_results import AnalyseResults
from mleap.data import Data
import pandas as pd

/media/viktor/Data/PhD/mleap/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='r')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')
analyze = AnalyseResults(hdf5_output_io=out_io, 
                        hdf5_input_io=input_io, 
                        input_h5_original_datasets_group='delgado_datasets/', 
                        output_h5_predictions_group='experiments/predictions/')
observations = analyze.calculate_error_per_dataset(metric='mean_squared_error')

#### t-test

In [ ]:
t_test, t_test_df = analyze.t_test(observations)
t_test_df

#### sign test

In [ ]:
sign_test, sign_test_df = analyze.sign_test(observations)
sign_test_df

#### t-test with Bonferroni correction

In [ ]:
t_test_bonferroni, t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(observations)
t_test_bonferroni_df

#### Wilcoxon test

In [ ]:
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(observations)
wilcoxon_test_df

#### Friedman test

In [ ]:
friedman_test, friedman_test_df = analyze.friedman_test(observations)
friedman_test_df

In [ ]:
nemeniy_test = analyze.nemenyi(observations)
pd.DataFrame(nemeniy_test)